Use file 'ar41_data.csv'.
Put the notebook and data file in one folder.
Run the entire notebook

In [2]:
# from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import folium
from scipy import stats

In [2]:
#drive.mount('/content/drive')
#path = '/content/drive/My Drive/BDMA/Semester 1 - ULB/Personal Work/Data Mining Project/ar41_data.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dtypes = {
    'mapped_veh_id': 'int32',
    'RS_E_InAirTemp_PC1': 'float32',
    'RS_E_InAirTemp_PC2': 'float32',
    'RS_E_OilPress_PC1': 'float32',
    'RS_E_OilPress_PC2': 'float32',
    'RS_E_RPM_PC1': 'float32',
    'RS_E_RPM_PC2': 'float32',
    'RS_E_WatTemp_PC1': 'float32',
    'RS_E_WatTemp_PC2': 'float32',
    'RS_T_OilTemp_PC1': 'float32',
    'RS_T_OilTemp_PC2': 'float32',
    'temp':'float32',
    'elevation':'float32'
}

data = pd.read_csv('ar41_data.csv', delimiter=',', dtype=dtypes)
data

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [7]:
data.drop(['Unnamed: 0', 'timestamps_UTC', 'weather',	'weather_description',	'elevation'], axis=1, inplace=True)

In [8]:
def calculate_speed(df, lat_col='lat', lon_col='lon', time_diff_col='time_diff_overall', veh_id_col='mapped_veh_id', journey_id_col='journey_id'):
    def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
        if to_radians:
            lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

        a = np.sin((lat2 - lat1) / 2.0)**2 + \
            np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2.0)**2

        return earth_radius * 2 * np.arcsin(np.sqrt(a))
    # Calculate the speed of a train in km/h between successive rows, grouped by train and journey_id.
    def calculate_group_speed(group):
        # Calculate the distance for each group
        group['distance'] = haversine(group[lat_col].shift(), group[lon_col].shift(), group[lat_col], group[lon_col])
        # Calculate speed in km/h, avoid division by zero
        group['speed'] = np.where(group[time_diff_col] > 0, (group['distance'] / group[time_diff_col]) * 3600, np.nan)
        # Handle the first entry of each group
        group['speed'].iloc[0] = 0
        group['distance'].iloc[0] = 0
        return group
    # Group by vehicle id and journey id, then apply the speed calculation
    df = df.groupby([veh_id_col], as_index=False).apply(calculate_group_speed)
    df = df.reset_index(drop=True)
    return df

def divide_journeys(data, time_threshold_1=300, time_threshold_2=600, speed_threshold = 0.5):
    condition = (data['time_diff_overall'] >= time_threshold_1) & \
                (data['speed'] < speed_threshold) | \
                (data['time_diff_overall'] >= time_threshold_2)
    data['time_diff'] = data['time_diff_overall'].where(~condition, 0).astype(int)
    data['new_journey'] = (data['time_diff'] == 0).astype(int)
    data['journey_id'] = data.groupby('mapped_veh_id')['new_journey'].cumsum()
    data.drop('new_journey', axis=1, inplace=True)

data = calculate_speed(data)
divide_journeys(data)

In [11]:
data.to_csv('journeys.csv', index=False)

In [1]:
import pandas as pd
data = pd.read_csv('journeys.csv')

KeyboardInterrupt: 

In [2]:
import pandas as pd

In [5]:
# data[data['mapped_veh_id'] == 162]
data[data['mapped_veh_id'] == 194]

,mapped_veh_id,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,RS_E_RPM_PC2,RS_E_WatTemp_PC1,...,RS_T_OilTemp_PC1,RS_T_OilTemp_PC2,temp,timestamps_Belgian,time_diff_overall,distance,speed,time_diff,journey_id,rolling_avg_speed
16933287,194,51.041244,3.683576,17.0,28.0,207.0,238.0,0.0,808.0,77.0,...,73.0,76.0,1.56,2023-01-23 09:28:12+00:00,0.0,0.000000,0.000000,0,1,0.127698
16933288,194,51.041278,3.683609,17.0,28.0,213.0,244.0,0.0,816.0,77.0,...,71.0,76.0,1.56,2023-01-23 09:29:14+00:00,62.0,0.004398,0.255396,62,1,0.127698
16933289,194,51.041239,3.683588,17.0,28.0,210.0,231.0,0.0,801.0,77.0,...,72.0,76.0,1.56,2023-01-23 09:30:15+00:00,61.0,0.004521,0.266795,61,1,0.261095
16933290,194,51.041247,3.683568,17.0,28.0,217.0,238.0,0.0,791.0,77.0,...,73.0,76.0,1.56,2023-01-23 09:31:15+00:00,60.0,0.001699,0.101943,60,1,0.184369
16933291,194,51.041239,3.683581,17.0,28.0,213.0,244.0,0.0,796.0,77.0,...,71.0,77.0,1.56,2023-01-23 09:32:15+00:00,60.0,0.001305,0.078296,60,1,0.090120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17141764,194,51.013760,3.779578,32.0,38.0,203.0,227.0,802.0,798.0,82.0,...,76.0,81.0,19.09,2023-09-13 19:39:19+00:00,50.0,0.002092,0.150633,50,432,0.651861
17141765,194,51.013759,3.779588,32.0,38.0,207.0,227.0,802.0,804.0,82.0,...,76.0,81.0,19.09,2023-09-13 19:39:30+00:00,11.0,0.000765,0.250217,11,432,0.200425
17141766,194,51.013762,3.779574,32.0,38.0,200.0,227.0,796.0,799.0,82.0,...,76.0,82.0,19.09,2023-09-13 19:40:29+00:00,59.0,0.001066,0.065017,59,432,0.157617
17141767,194,51.013464,3.780101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,77.0,82.0,19.09,2023-09-13 19:41:30+00:00,61.0,0.049583,2.926191,61,432,1.495604


In [16]:
# data.to_csv('train162.csv', index=False)

# data[data['mapped_veh_id'] == 162].to_csv('train162.csv', index=False)

In [4]:
data = pd.read_excel('train162.xlsx')

In [6]:
data[data['mapped_veh_id'] == 194].to_excel('train194.xlsx', index=False)

In [1]:
# data[data['mapped_veh_id'] == 194].to_csv('train194.csv', index=False)

NameError: name 'data' is not defined

In [1]:
import pandas as pd

In [9]:
columns_to_check = ['RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC2', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC2']
data.dropna(subset=columns_to_check, inplace=True)

In [10]:
def rolling_operation(x):
    return x.rolling(window=2).mean().bfill().ffill()
data['rolling_avg_speed'] = data.groupby(['mapped_veh_id', 'journey_id'])['speed'].transform(rolling_operation)
data['rolling_avg_speed'] = data['rolling_avg_speed'].fillna(data['speed'])

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

check = data.head(100000)
# Assuming 'data' is your DataFrame and includes all the necessary columns
# Preprocessing: Select relevant features and normalize
features = check[['RS_E_InAirTemp_PC1', 'RS_E_OilPress_PC1', 'RS_E_RPM_PC1', 'RS_E_WatTemp_PC1', 'RS_T_OilTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC2', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC2', 'rolling_avg_speed']]
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

# Instantiate and fit the Isolation Forest model
model = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', random_state=42)
model.fit(features_normalized)

# Predict anomalies
check['anomaly'] = model.predict(features_normalized)
check['anomaly'].value_counts()

C:\Users\Hareem Raza\AppData\Local\Temp\ipykernel_10320\1559081916.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['anomaly'] = model.predict(features_normalized)


anomaly
 1    7579
-1    2421
Name: count, dtype: int64

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Assuming 'data' is your DataFrame and 'check' is a subset of the data
# Preprocessing: Select relevant features and normalize
features =  check[['RS_E_InAirTemp_PC1', 'RS_E_OilPress_PC1', 'RS_E_RPM_PC1', 'RS_E_WatTemp_PC1', 'RS_T_OilTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC2', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC2', 'rolling_avg_speed']]
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

# Instantiate and fit the DBSCAN model
# Note: You need to choose appropriate values for eps and min_samples
dbscan = DBSCAN(eps=0.2, min_samples=10)
clusters = dbscan.fit_predict(features_normalized)

# Mark anomalies (DBSCAN labels outliers as -1)
check['anomaly_dbscan'] = (clusters == -1).astype(int)
# Review the results
print(check['anomaly_dbscan'].value_counts())

anomaly_dbscan
0    6226
1    3774
Name: count, dtype: int64


C:\Users\Hareem Raza\AppData\Local\Temp\ipykernel_10320\1958940382.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['anomaly_dbscan'] = (clusters == -1).astype(int)


In [ ]:
# Set the timestamp as the index
check.set_index('timestamps_Belgian', inplace=True)
# Sort the data by the index (timestamp)
check.sort_index(inplace=True)
# Select the feature for ARIMA modeling, e.g., 'speed'
time_series = check[['RS_E_InAirTemp_PC1', 'RS_E_OilPress_PC1', 'RS_E_RPM_PC1', 'RS_E_WatTemp_PC1', 'RS_T_OilTemp_PC1', 'rolling_avg_speed']]

## VARIMA

In [ ]:
from sklearn.preprocessing import StandardScaler

features =  check[['RS_E_InAirTemp_PC1', 'RS_E_OilPress_PC1', 'RS_E_RPM_PC1', 'RS_E_WatTemp_PC1', 'RS_T_OilTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC2', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC2', 'rolling_avg_speed']]
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

In [ ]:
from statsmodels.tsa.api import VARMAX
# Convert normalized features to DataFrame
features_df = pd.DataFrame(features_normalized, columns=features.columns, index=check.index)
# Fit VARIMA model
# Note: Determine the order (p, d, q) appropriately; here, we use (1,1,1) as an example
model = VARMAX(features_df, order=(1, 1), trend='c').fit(disp=False)

C:\Python310\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Python310\lib\site-packages\statsmodels\tsa\statespace\varmax.py:326: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  endog = np.require(endog.fillna(method='backfill'), requirements="W")


In [ ]:
# Get residuals
residuals = model.resid

# Anomaly detection - mark as anomaly if residuals for any variable are beyond a threshold
anomaly_threshold = 2.5  # Define based on your understanding of the data
check['anomaly'] = residuals.apply(lambda x: np.any(np.abs(x) > anomaly_threshold), axis=1).astype(int)

In [ ]:
# Check anomaly counts
print(check['anomaly'].value_counts())
# Optionally, inspect some of the anomalies
print(check[check['anomaly'] == 1])